In [1]:
import numpy as np
import sklearn.datasets
import sklearn.metrics
import xgboost as xgb
import optuna

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import cryptoaml.datareader as cdr
from cryptoaml.models import XgboostAlgo

/home/delinvas/anaconda3/envs/btc-classifier/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.



In [2]:
elliptic = cdr.get_data("elliptic")
data = elliptic.train_test_split(train_size=0.7, feat_set="LF")

In [3]:
train_X = data.train_X
train_y = data.train_y

test_X = data.test_X
test_y = data.test_y

In [11]:
tmp_estimator = XgboostAlgo()
def objective(trial):
    
    param = {
        # using RS
        # "learning_rate": trial.suggest_discrete_uniform("learning_rate", 0.05, 0.3, 0.025),       
#         "n_estimators": trial.suggest_int("n_estimators", 100, 800, 25),
        
        
#         "tree_method":"gpu_hist", 
#         "predictor":"gpu_predictor"
        
        "max_depth": trial.suggest_int("max_depth", 1, 12),
        
        "subsample": trial.suggest_uniform("subsample", 0.9, 1.0),
        
        
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.9, 1.0),  
        
        
        "min_child_weight": trial.suggest_int("min_child_weight", 0, 4),
        
        
        "max_delta_step": trial.suggest_int("max_delta_step", 0, 1),
        
        "gamma": trial.suggest_discrete_uniform("gamma", 0.0, 0.01, 0.005)       
        
#         "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 1.0),
#         "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 1.0)
    }

#     if param["learning_rate"] < 0.1:
#         param["n_estimators"] = trial.suggest_int("n_estimators", 400, 800, 25)
#     else: 
#         param["n_estimators"] = trial.suggest_int("n_estimators", 100, 400, 25)

    
    tmp_estimator.set_params(**param)
    scores = cross_val_score(tmp_estimator, 
                                 train_X, 
                                 train_y, 
                                 scoring="neg_log_loss", 
                                 cv=StratifiedKFold(n_splits=3),
                                 n_jobs=1)
    
    print(tmp_estimator.get_params())
    mean_score = scores.mean()
    trial.set_user_attr("cv_mean", mean_score)    
    std_score  = scores.std()
    trial.set_user_attr("cv_std", std_score)
    min_score  = scores.min()
    trial.set_user_attr("cv_min", min_score)
    max_score  = scores.max()
    trial.set_user_attr("cv_max", max_score)

    return mean_score

# study = optuna.create_study(sampler=optuna.samplers.TPESampler(), 
#                             direction="maximize")



study = optuna.create_study(sampler=optuna.samplers.TPESampler(), 
                            direction="maximize")


# study = optuna.create_study(sampler=optuna.samplers.GridSampler({
#     "n_estimators": [100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400]}), 
#                             direction="maximize")

study.set_user_attr("k_folds", 3)
study.set_user_attr("cv_method", "StratifiedKFold")
# study.optimize(objective, n_trials=10)
study.optimize(objective, n_trials=100, n_jobs=1)

{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.9075247415183287, 'gamma': 0.005, 'gpu_id': None, 'importance_type': 'gain', 'interaction_constraints': None, 'learning_rate': None, 'max_delta_step': 1, 'max_depth': 9, 'min_child_weight': 3, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': 1.4368047841751952e-08, 'reg_lambda': 0.08206186516337204, 'scale_pos_weight': None, 'subsample': 0.9130310112681479, 'tree_method': None, 'validate_parameters': False, 'verbosity': None}


[I 2020-04-08 02:38:05,340] Finished trial#0 with value: -0.45419105224053347 with parameters: {'max_depth': 9, 'subsample': 0.9130310112681479, 'colsample_bytree': 0.9075247415183287, 'min_child_weight': 3, 'max_delta_step': 1, 'gamma': 0.005, 'reg_lambda': 0.08206186516337204, 'reg_alpha': 1.4368047841751952e-08}. Best is trial#0 with value: -0.45419105224053347.


{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 0.9524850833087666, 'gamma': 0.0, 'gpu_id': None, 'importance_type': 'gain', 'interaction_constraints': None, 'learning_rate': None, 'max_delta_step': 1, 'max_depth': 11, 'min_child_weight': 2, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': 1.7916932398213714e-05, 'reg_lambda': 0.1908007677218535, 'scale_pos_weight': None, 'subsample': 0.9806004744510634, 'tree_method': None, 'validate_parameters': False, 'verbosity': None}


[I 2020-04-08 02:38:26,249] Finished trial#1 with value: -0.514252044910191 with parameters: {'max_depth': 11, 'subsample': 0.9806004744510634, 'colsample_bytree': 0.9524850833087666, 'min_child_weight': 2, 'max_delta_step': 1, 'gamma': 0.0, 'reg_lambda': 0.1908007677218535, 'reg_alpha': 1.7916932398213714e-05}. Best is trial#0 with value: -0.45419105224053347.


KeyboardInterrupt: 

In [ ]:
# display(optuna.visualization.plot_optimization_history(study))
print(study.best_trial)
display(study.trials_dataframe())

In [ ]:
# optuna.visualization.is_available()

In [ ]:
tmp_estimator.set_params(**study.best_trial.params)

print(tmp_estimator.get_params())

tmp_estimator.fit(train_X, train_y)

In [ ]:
results = tmp_estimator.evaluate(["f1"], test_X, test_y)

In [ ]:
print(results)

# 0.7820773930753565
# 0.7820773930753565

In [ ]:
print(tmp_estimator.get_params())

In [ ]:
display(study.trials_dataframe())

In [ ]:
# save study
import pickle

with open("gs_xgboost_LF.pkl", "wb") as model_file:
    pickle.dump(study, model_file)



In [ ]:
# display(train_X)